Compare the derived solutions with the exact Gauss-Germite polynomial solutions for the quantum harmonic oscillator PDE. The error will be given by  
$$
	\left\|\left|\phi_n^{\text {numerical }}\right|-\left|\phi_n^{\text {exact }}\right|\right\|
$$
where $\|f(x)\|=\int_{-L}^L f(x)^2 d x$. 

For the eigenvalues, we will simply calculate the relative percent error 
$$
	100 \times\left(\left|\varepsilon_n^{\text {numerical }}-\varepsilon_n^{\text {exact }}\right| / \varepsilon_n^{\text {exact }}\right).
$$

In [9]:
# Imports
import math
import numpy as np
from scipy.special import hermite
from scipy.integrate import quad

In [10]:
# Parameters
K = 1   
L = 4   
n_modes = 5  
xs = np.linspace(-L, L, 81)  

In [11]:
# Setup
eigvecs_exact = []
eigvals_exact = np.array([2*n + 1 for n in range(n_modes)])

# Gauss-Hermite polynomial function
def hermite_function(x, n):
    return np.exp(-x**2 / 2) * hermite(n)(x) / np.sqrt(2**n * math.factorial(n) * np.sqrt(np.pi))

# Compute eigenfunctions for n_modes
for n in range(n_modes):
    psi_ns = hermite_function(xs, n)
    norm = np.sqrt(quad(lambda x: hermite_function(x, n)**2, -L, L)[0])
    psi_n_normed = psi_ns / norm
    eigvecs_exact.append(psi_n_normed)

eigvecs_exact = np.array(eigvecs_exact).T

In [12]:
## Using shooting

# Imports
import numpy as np
from scipy.integrate import odeint

# Define bvp
def ansatz_bvp(y, x, epsilon):
	return [y[1], (x**2 - epsilon) * y[0]]

# Define parameters
tol = 1e-6							# Tolerance
L = 4 								# Boundary given in problem

xp = [-L, L]								# Boundary values
samples = int(2 * L / 0.1) + 1				# Number of samples for linspace; np doesn't like float
xspan = np.linspace(xp[0], xp[1], samples)	# Range of x to search through -- boundaries

# Shooting

# Define our initial conditions
A = 0.1 				# Initial value for dpsi -- can be anything except 0
initial_epsilon = 0		# Initial value for epsilon

# Cumulator
eigenvalues = []
eigenfunctions = []

# Find first 5 eigenfunctions
for modes in range(5):
	epsilon = initial_epsilon			# Start search for epsilon
	depsilon = 0.01						# Change in epsilon
	dp0 = A								# Initial dpsi value
	p0 = dp0 / np.sqrt(L**2 - epsilon)	# Initial psi value
	psi0 = [p0, dp0]					# Initial values for psi and dpsi 
	
	for j in range(1000):	# Find epsilon in 1000 steps max; fail safe
		sol = odeint(ansatz_bvp, psi0, xspan, args=(epsilon,))	# Solve ODE for current epsilon

		if abs(sol[-1, 0] / p0) < tol:	# If found solution is below tolerance
			eigenvalues.append(epsilon)		# Save eigenvalue
			break							# Stop searching; move onto next eigenvalue

		if (-1) ** (modes) * (sol[-1, 0] / p0) > 0:	# If below target
			epsilon = epsilon + depsilon 	# Increase epsilon
		else:									# Otherwise above target
			epsilon = epsilon - depsilon/2	# Binary search for epsilon
			depsilon = depsilon/2			# Binary search for epsilon

	initial_epsilon = epsilon + 0.1	# Increase epsilon; search for next eigenvalue

	y_norm = np.trapz(sol[:, 0]**2, xspan)		# Normalize Schrodinger pdf
	y_solution = sol[:, 0] / np.sqrt(y_norm)	# Divide by norm
	eigenfunctions.append(y_solution)			# Save eigenfunction

ef_shoot = np.column_stack(np.abs(eigenfunctions))
ev_shoot = np.vstack(eigenvalues)

In [13]:
## Using direct solve

import numpy as np
from scipy.sparse import diags
from scipy.sparse.linalg import eigs

# Defined parameters
num_eigenvalues= 5

L = 4
K = 1
dx = 0.1

N = int((L - -L)/dx) + 1
xspan = np.linspace(-4, 4, N)


# Direct solve

# Construct diagonals
main_diagonal = 2*np.ones(N) / dx**2
upper_diagonal = -np.ones(N - 1) / dx**2
lower_diagonal = -np.ones(N - 1) / dx**2

# Found boundary values
main_diagonal[0] *= 4/3
main_diagonal[-1] *= 4/3

upper_diagonal[0] *= -1/3
lower_diagonal[-1] *= -1/3

# Add X transformation to main diagonal
X = K * xspan**2
main_diagonal += X

# Construct derivative matrix 
D = diags([lower_diagonal, main_diagonal, upper_diagonal], [-1, 0, 1])

# Find eigenvalues and eigenvectors
eigenvalues, eigenvectors = eigs(D, k=num_eigenvalues, which='SM')
eigenvalues = eigenvalues.real
eigenvectors = np.abs(eigenvectors.real)

ef_direct = eigenvectors
ev_direct = eigenvalues

In [14]:
L = 4
xs = np.linspace(-L, L, ef_shoot.shape[0])  
n_modes = 5   

err_ef_shoot = [ 
	np.trapz(( np.abs(ef_shoot[:, i]) - np.abs(eigvecs_exact[:, i]) )**2, xs) 
	for i in range(n_modes) 
]

err_ev_shoot = np.array([
	[ 100 * np.abs( (ev_shoot[i] - eigvals_exact[i]) / eigvals_exact[i]) ] 
	for i in range(n_modes)
]).squeeze()

err_ef_direct = [np.trapz(
	( np.abs(ef_direct[:, i]) - np.abs(eigvecs_exact[:, i]))**2, xs ) 
	for i in range(n_modes)
]

err_ev_direct = np.array([
	[100 * np.abs((ev_direct[i] - eigvals_exact[i]) / eigvals_exact[i])] 
	for i in range(n_modes)
]).squeeze()

display(err_ef_shoot)
display(err_ev_shoot)
display(err_ef_direct)
display(err_ev_direct)


[9.282651584716013e-09,
 3.2586450316320717e-07,
 5.558070056782005e-06,
 6.253600138479176e-05,
 0.0005233314439583922]

array([4.98064329e-05, 4.94078810e-04, 4.08833981e-03, 2.45410272e-02,
       1.08285048e-01])

[0.4675445409379586,
 0.4675452485980933,
 0.4675509320467866,
 0.46759823275627554,
 0.46791990783479]

array([0.0624279 , 0.10320785, 0.15407958, 0.17260434, 0.0632553 ])